# Regresszió

Szükséges csomagok beolvasása:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

## Egyváltozós lineáris regresszió

Az egyváltozós lineáris regresszió bemutatásához egy egyszerű adathalmazt használunk, amely azt tartalmazza, hogy egy hallgató mennyi időt töltött a tanulással (*Hours*)) és a zh eredménye hány százalékos lett (*Scores*). 

Az adathalmaz kevés mintát tartalmaz, de a bemutatáshoz számunkra most ez is elegendő lesz.

In [ ]:
dataset = pd.read_csv('student_scores.csv')
dataset.head(10)

Egy gyors betekintés az adatokba.

In [ ]:
#attribútumok alapstatisztikai adatai:
dataset.describe()

In [ ]:
dataset.plot(x='Hours', y='Scores', style='o')
plt.title('Hours vs Percentage')
plt.xlabel('Hours Studied')
plt.ylabel('Percentage Score')
plt.show()

**Adatelőkészítés (függő és független változók, tréning és teszt halmazok kialakítása)**

In [ ]:
# Függő és független változók:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 1].values

# Test és training adatok létrehozása
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

**Modell példányosítása és tanítása**

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

**Az illesztett modell paraméterei**

In [ ]:
print ('b0 értéke:', regressor.intercept_)
print ('b1 értéke:', regressor.coef_)

Értelmezése: minden tanulással töltött óra 9,91% százalékkal növeli a zh eredményét.

**Előrejelzés a modell alapján**

In [ ]:
# Előrejelzés a teszt halmazon
y_pred = regressor.predict(X_test)

ds_prediction = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
ds_prediction

**Az illesztett modell hibája**

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

**Modell kirajzolása**

In [ ]:
# Plot outputs
plt.scatter(X_train, y_train,  color='green', label='Train data')
plt.scatter(X_test, y_test,  color='red', label='Test data')
plt.plot(X, regressor.predict(X), color='blue', linewidth=3, label='Linear regression model')

plt.legend()

plt.title('Hours vs Percentage')
plt.xlabel('Hours Studied')
plt.ylabel('Percentage Score')

plt.show()

## Többváltozós lineáris regresszió

Példa adathalmaz:

Az USA-ban 48 államban megmérték az egy éves benzin fogyasztást. A releváns változók: a benzinadó, az átlagos jövedelem, az aszfaltozott autópálya mérföldjeinek száma, és a vezetői engedéllyel rendelkező lakosság aránya.

Cél: a benzinfogyasztás előrejelzése

Attribútumok:
- Petrol_tax: benzinadó (cent/gallon)
- Average_income: 1 főre eső átlagos jövedelem (dollár);
- Paved_highways: aszfaltozott autópályák hossza (mérföld);
- Population_Driver_licence(%): jogosítványok aránya;
- Petrol_Consumption: benzinfogyasztás (millió gallon)

Az adathalmaz beolvasása és egy kis betekintés:

In [ ]:
dataset = pd.read_csv('petrol_consumption.csv')
dataset.head()

In [ ]:
dataset.describe()

**Adatelőkészítés (függő és független változók, tréning és teszt halmazok kialakítása)**

In [ ]:
# Függő és független változók:
X = dataset[['Petrol_tax', 'Average_income', 'Paved_Highways',
       'Population_Driver_licence(%)']]
y = dataset['Petrol_Consumption']

# Test és training adatok létrehozása
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

**Modell példányosítása és tanítása**

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

**Az illesztett modell paraméterei**

In [ ]:
df_coeff = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
df_coeff

Értelmezése: 

Az üzemanyag adó (Petrol_tax) egységnyi növekedése esetén 32 millió gallonnal csökken a benzinfogyasztás. A járművezetői engedéllyel rendelkező lakosság arányának egységnyi növekedése 1,355 milliárd liter többlet benzinfogyasztást eredményez. Az átlagos jövedelem (Average_income) és az aszfaltozott útak (Paved_highways) változása nagyon csekély hatással van a benzinfogyasztásra.

**Előrejelzés a modell alapján**

In [ ]:
# Előrejelzés a teszt halmazon
y_pred = regressor.predict(X_test)

df_prediction_MLR = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_prediction_MLR

**Az illesztett modell hibája**

In [ ]:
from sklearn import metrics

MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)
print('Root Mean Squared Error:', RMSE)

Mint láthatjuk, az átlagos abszolút hiba 41,92, ami az összes állam átlagos fogyasztásának (576,77) a 7,2%-a. A hiba nem kicsi, de talán még elfogadható. Ugyanez az RMSE-re vetítve 9,0%.

Mi okozhatja a hibát?
- Feltételeztük a lineáris összefüggést. Tényleg igaz?
- Kevés a megfigyelt adat (csak 1 évnyi). Több évnyi adat pontosabban mutatná az összefüggéseket.
- Lehet nem is ezek a független változók jelzik előre a benzinfogyasztást...

## __Lasso, Ridge, Elastic Net__

**Ridge regresszió**

In [ ]:
from sklearn.linear_model import Ridge

# alpha: súlyozási tényező a regulariziációs taghoz
regressor_Ridge = Ridge(alpha=0.1)
regressor_Ridge.fit(X_train, y_train)

#együtthatók:
df_coeff['Coefficient_Ridge'] = regressor_Ridge.coef_

# Előrejelzés a teszt halmazon
y_pred_Ridge = regressor_Ridge.predict(X_test)

df_prediction_MLR['Predicted_Ridge'] = y_pred_Ridge

MAE_Ridge = metrics.mean_absolute_error(y_test, y_pred_Ridge)
MSE_Ridge = metrics.mean_squared_error(y_test, y_pred_Ridge)
RMSE_Ridge = np.sqrt(metrics.mean_squared_error(y_test, y_pred_Ridge))

**Lasso regresszió**

In [ ]:
from sklearn.linear_model import Lasso

# alpha: súlyozási tényező a regulariziációs taghoz
regressor_Lasso = Lasso(alpha=0.1)
regressor_Lasso.fit(X_train, y_train)

#együtthatók:
df_coeff['Coefficient_Lasso'] = regressor_Lasso.coef_

# Előrejelzés a teszt halmazon
y_pred_Lasso = regressor_Lasso.predict(X_test)

df_prediction_MLR['Predicted_Lasso'] = y_pred_Lasso

MAE_Lasso = metrics.mean_absolute_error(y_test, y_pred_Lasso)
MSE_Lasso = metrics.mean_squared_error(y_test, y_pred_Lasso)
RMSE_Lasso = np.sqrt(metrics.mean_squared_error(y_test, y_pred_Lasso))

**Elastic Net**

In [ ]:
from sklearn.linear_model import ElasticNet

# alpha: súlyozási tényező a regulariziációs tagokhoz
#l1_ratio: [0,1] közti arány az L1 és L2 regularizációhoz 
#(l1_ratio=0: csak L2 bűntetés van; l1_ratio=1: csak L1 bűntetés van)
regressor_ElasticNet = ElasticNet(alpha=0.1, l1_ratio=0.5)
regressor_ElasticNet.fit(X_train, y_train)

#együtthatók:
df_coeff['Coefficient_ElasticNet'] = regressor_ElasticNet.coef_

# Előrejelzés a teszt halmazon
y_pred_ElasticNet = regressor_ElasticNet.predict(X_test)

df_prediction_MLR['Predicted_ElasticNet'] = y_pred_ElasticNet

MAE_ElasticNet = metrics.mean_absolute_error(y_test, y_pred_ElasticNet)
MSE_ElasticNet = metrics.mean_squared_error(y_test, y_pred_ElasticNet)
RMSE_ElasticNet = np.sqrt(metrics.mean_squared_error(y_test, y_pred_ElasticNet))

Regressziós együtthatók kiíratása:

In [ ]:
df_coeff

Előrejelzések:

In [ ]:
df_prediction_MLR

In [ ]:
df_errors = pd.DataFrame([['Simple', MAE, MSE, RMSE],
                          ['Ridge', MAE_Ridge, MSE_Ridge, RMSE_Ridge], 
                          ['Lasso', MAE_Lasso, MSE_Lasso, RMSE_Lasso], 
                          ['Elastic Net', MAE_ElasticNet, MSE_ElasticNet, RMSE_ElasticNet]],
                         columns=(['Method', 'MAE', 'MSE', 'RMSE']))
df_errors

## __Logisztikus regresszió__

A Scikit-Learn *LogisticRegression* osztálya alapesetben regularizált logisztikus regressziót valósít meg. 

A 3 legfontosabb paraméter:
- penalty {'l1', 'l2', 'elasticnet', 'none'}, default='l2': regularizáció L1 és L2 C értékét befolyásolja, L2 esetén C értéket négyzetesen befolyásolja.
- C: koefficiens [0,1]-ből vesz fel értéket. Kisebb érték, nagyobb regularizációs megszorításokat jelent, míg a nagyobb érték, nagyobb szabadság a modell számára.
- solver: a megoldó - Legtöbb esetben a legjobb megoldó a Scikit learn szerint a SAGA.
[Solver részletesebb bemutatása](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)






In [ ]:
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import plot_confusion_matrix

Titanic adathalmaz importálása és előkészítése:

In [ ]:
# DataFrame létrehozása Excel fájlból
titanic = pd.read_excel('Titanic.xlsx')

# Eldobjuk a NAME oszlopot
titanic.drop(labels='Name', axis=1, inplace=True)

# Dummy váltóvá konvertáljuk a SEX oszlopot (referencia kódolással, ezért csak 1 oszlop lesz belőle)
titanic['Sex'] = pd.get_dummies(titanic['Sex'], drop_first=True)

# a prediktív válltozók neveinek és az osztálycímke kimentése
classlabel = list(titanic.columns[:1])
features = list(titanic.columns[1:])

# DataFrame-et TULAJDONSÁGOK(features, X) és CÉL(target, y) tömbökre bontjuk 
X = titanic.iloc[:, 1:].values
y = titanic.iloc[:, 0].values.reshape(-1,1)

# adatok standardizálása a StandardScaler-rel (mivel a logisztikus regresszió nagyon érzények a skálára)
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

Tréning/teszt adatok kialakítása:

In [ ]:
# Tréning és teszt adatok létrehozása 2/3 és 1/3 vágással
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

#### A modell illesztése az alapparaméterekkel

In [ ]:
clf_logit = LogisticRegression()
clf_logit.fit(X_train, y_train.ravel())
print("A modell pontossága:", clf_logit.score(X_test,y_test))

In [ ]:
plot_confusion_matrix(clf_logit, X_test, y_test, cmap=plt.cm.Blues)
plt.show()